# Extract Max Amplitudes
We provide documentation for the extraction of maximum amplitudes using both our day-stacks and for time-series analysis and from our yearly stacks for an aggregate maximum amplitude signal. The codes below are also useful for indexing and extracting correlations.

In [11]:
# packages
using SeisIO, SeisNoise, Plots, Dates, CSV, DataFrames, JLD2, Statistics, Glob, AbstractFFTs, HDF5

In [2]:
# Functions for correlation extraction and processing: Takes jld2 and HDF5 files to SeisData Objects
function fftderivative(A::CorrData)
    C = deepcopy(A)
    detrend!(C)
    taper!(C)
    F = fft(C.corr,1)
    F .*= fftfreq(length(C.corr)).* 1im .* 2π
    C.corr = real.(ifft(F,1))/(C.fs^2)
    return C
end
# gets location and maximum amplitude for corrs in array
function loc_max_amp(corr::CorrData, filt::Array{Float64,1}=[0.1,0.5])
    lat, lon = corr.loc.lat, corr.loc.lon
    bpass_corr = bandpass(fftderivative(fftderivative(corr)),filt[1],filt[2])
    half_len = convert(Int64,round(length(bpass_corr.corr)/2))
    max_ampl = maximum(bpass_corr.corr)
    max_ampl_anti = maximum(bpass_corr.corr[1:half_len])
    max_ampl_causal = maximum(bpass_corr.corr[half_len:end])
    snr = maximum(bpass_corr.corr)/std(bpass_corr.corr)
    snr_anti = maximum(bpass_corr.corr[1:half_len])/std(bpass_corr.corr[1:half_len])
    snr_causal = maximum(bpass_corr.corr[half_len:end])/std(bpass_corr.corr[half_len:end])
    return (lat, lon, max_ampl, max_ampl_anti, max_ampl_causal, snr, snr_anti, snr_causal, corr.name)
end
function create_corr(file::HDF5File, station::String, stacktype::String, component::String)
    try
        S = CorrData()
        # get corr data
        corr = file[station][component][stacktype][1:end]
        S.corr = reshape(corr,size(corr,1),1)
        S.name = station
        S.dist = read(file[station]["meta"]["dist"])
        S.loc.lat = read(file[station]["meta"]["lat"])
        S.loc.lon = read(file[station]["meta"]["lon"])
        S.loc.el = read(file[station]["meta"]["el"])
        S.azi = read(file[station]["meta"]["azi"])
        S.baz = read(file[station]["meta"]["baz"])
        S.fs = 20. # For the 20 Hz product
        return S
    catch e
        #println(e)
    end
end

# get information for a set of filtered stations, default to all stations
function get_corrs(file::HDF5File, stacktype::String="linear",
    component::String="ZZ", filter::String="")
    filtered = [name for name in names(file) if occursin(filter, name)] 
    corrs = [create_corr(file, station, stacktype, component) for station in filtered]
    corrs = [corr for corr in corrs if !isnothing(corr)] # strip nothings - catches meta file etc which errors
    return corrs
end 

get_corrs (generic function with 4 methods)

In [20]:
year1, year2, year3 = 2017, 2018, 2019 # large stacked files have data between 2017 and 2019
filt = "" # return nodal plots
bandpass_filter, stacktype, components = [0.1, 0.35], "linear", ["ZZ","NN","EE"]
rootdir = "/Users/julianschmitt/Downloads/Seismo/"
for source_station in ["CHN","CJM","DEV","IPT","LPC","SNO","SVD","TA2"]
    fpath = joinpath(rootdir,"processed/$(year1)/$(year1)_CI.$(source_station).h5")
    f2017 = h5open(fpath,"r")
    f2018 = h5open(joinpath(rootdir,"processed/$(year2)/$(year2)_CI.$(source_station).h5"),"r")
    f2019 = h5open(joinpath(rootdir,"processed/$(year3)/$(year3)_CI.$(source_station).h5"),"r")
    for comp in components
        all_corrs = collect(Iterators.flatten([get_corrs(file, stacktype, comp, filt) for file in [f2017, f2018, f2019]]))
        amplitudes = [loc_max_amp(corr, bandpass_filter) for corr in all_corrs]
        #pgv_amp = DataFrame(amplitudes, ["lat", "lon", "max_amp", "max_amp_anti", "max_amp_causal", "SNR", "SNR_anticausal", "SNR_causal", "station"])#[:lat, :lon, :max_amp, :max_amp_anti, :max_amp_causal, :SNR, :SNR_anticausal, :SNR_causal, :station])
        # if there's a better way to make this dataframe like ^^ please please please tell me :) 
        pgd_amp = DataFrame(lat = [x[1] for x in amplitudes], lon = [x[2] for x in amplitudes],
            max_amp = [x[3] for x in amplitudes], max_amp_anti = [x[4] for x in amplitudes], 
            max_amp_causal = [x[5] for x in amplitudes],SNR = [x[6] for x in amplitudes], 
            SNR_anticausal = [x[7] for x in amplitudes], SNR_causal = [x[8] for x in amplitudes], station = [x[end] for x in amplitudes]);
        pgv_grouped = aggregate(groupby(pgd_amp, :station), mean)
        #println(pgd_amp)
        save_path = joinpath(rootdir,"amplitude_data/PGV_0.1_0.35/PGV_$(source_station)_$comp.csv")
        if !isdir(dirname(save_path))
            mkpath(dirname(save_path))
        end
        CSV.write(save_path ,pgv_grouped)
    end
end

HDF5-DIAG: Error detected in HDF5 (1.12.0) thread 0:
  #000: H5O.c line 136 in H5Oopen(): unable to open object
    major: Object header
    minor: Can't open object
  #001: H5VLcallback.c line 5418 in H5VL_object_open(): object open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 5385 in H5VL__object_open(): object open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_object.c line 60 in H5VL__native_object_open(): unable to open object by name
    major: Object header
    minor: Can't open object
  #004: H5Oint.c line 649 in H5O_open_name(): object not found
    major: Object header
    minor: Object not found
  #005: H5Gloc.c line 462 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #006: H5Gtraverse.c line 855 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #007: H5Gtraverse.c line 630 in H5G__travers

## Daily Max Amplitudes to CSV
We now focus on computing time series of daily averages for plots using corr_large files 

In [ ]:
all_node_files = glob("nodes/corr_large/*/*/*")
set_nodes = Set([split(f, "/")[end] for f in all_node_files])
for node_files in set_nodes
    try
        raw_files = glob("nodes/corr_large/*/*/$node_files")
        if length(raw_files) ==0
            continue
        end
        source, reciever = join(split(node_files,".")[1:2],"."), join(split(node_files,".")[4:5],".")
        println(source,"  ", reciever, " had $(length(raw_files)) files out of 36.")
        amplitudes = Array{Any, 1}(undef, 0)
        times = Array{Date, 1}(undef, 0)
        for file in raw_files 
            A = jldopen(file, "r")
            for key in keys(A["ZZ"])
                corr = A["ZZ/$key"]
                attrs = loc_max_amp(corr) 
                push!(amplitudes,attrs)
                push!(times, Date(key))
            end
        end
        # sort by date
        p = sortperm(times)
        times = times[p]
        amplitudes = amplitudes[p]

        # name and plot
        name_ar = join(deleteat!(split(split(raw_files[1],"/")[end],"."), [3,6]),".")
        tseries_data = DataFrame(date = times, lat = [x[1] for x in amplitudes], lon = [x[2] for x in amplitudes],
            max_amp = [x[3] for x in amplitudes], max_amp_anti = [x[4] for x in amplitudes], 
            max_amp_causal = [x[5] for x in amplitudes],SNR = [x[6] for x in amplitudes], 
            SNR_anticausal = [x[7] for x in amplitudes], SNR_causal = [x[8] for x in amplitudes], station = [x[end] for x in amplitudes]);
        save_path = "/Users/julianschmitt/Downloads/Seismo/amplitude_data/Tseries_nodes/$(name_ar).csv"
        CSV.write(save_path, tseries_data)
    catch e
        println(e)
    end
end

### Plotting Routines 

In [25]:
function mov_avg(df::DataFrame, range::Int64=30) 
    df_new = df[:,["date","max_amp"]]
    amps = df_new.max_amp
    avg, len = similar(amps), length(amps)
    for i =1:len
        first = max(1, convert(Int64, round(i-range/2)))
        last = min(len, convert(Int64, round(i+range/2)))
        window = amps[first:last]
        sd, mn = std(window), mean(window)
        avg[i] = mean(filter(x -> (x < mn + 3*sd) && (x > mn-3*sd), window))
    end
    df_new.moving_avg = avg
    return df_new
end
function get_node_files(source::String, deployment_filt::String, int_range=Array{Int64, 1})
    return filter(x -> occursin(source, x) & occursin(deployment_filt,x) & (parse(Int64, split(x, ".")[end-2][end-1:end]) in collect(int_range[1]:int_range[2])), nodes)
end
# Plotting and wrapper function to select nodes to plot
plot_root = "/Users/julianschmitt/Desktop/SeisPlots2/"
function plot_tseries(perm_station::String, node_stations_::Array{String,1}, source_, dir::String)
    test = DataFrame(CSV.File(perm_station))
    #get moving averages and naming schemes
    times, amplitudes = test.date, test.max_amp
    times_avg, amps_avg = mov_avg(test,90).date, mov_avg(test,90).moving_avg
    perm_name = join(split(split(perm_station,"/")[end],".")[3:4],".")
    node_range = join([split(node_stations_[1],".")[end-2],split(node_stations_[end],".")[end-2][end-1:end]],"-")
    name = "$perm_name and $node_range Peak Amplitude Time Series"
    
    # Plot permanent station moving average
    tseries = plot(times_avg, amps_avg, title = name, label="$perm_name Mov Avg", color="gray", 
        linewidth=2, xtickfontsize=10, xlabel="Time", ylabel="Correlation Velocity Max Amp - filt: 0.1-0.5 Hz",
        legend = :outertopright, size=(900, 400), dpi=300, leftmargin= 7mm, bottommargin=5mm, linealpha=.6)
    
    # plot nodes
    for (ind, file) in enumerate(node_stations_)
        node_name, color = join(split(split(file,"/")[end],".")[3:4],"."), ""
        if occursin("NO.B1", node_name) # color B1 nodes green and intersecting nodes blue
            color = "green"
        else
            color = "blue"
        end
        test2 = DataFrame(CSV.File(file))
        test3 = mov_avg(test2, 20)
        times2, amplitudes2 = test2.date, test2.max_amp
        times3, amplitudes3 = test3.date, test3.moving_avg
        plot!(tseries, times3, amplitudes3, color=color, linealpha=.6, linewidth=2, label="$node_name Mov Avg")
    end
    save_path = joinpath(plot_root, "tseries_combined/$dir/CI.$(source_)_$(perm_name)_to_nodes.png")
    if !isdir(dirname(save_path))
        mkpath(dirname(save_path))
    end
    png(save_path)
end
function plot_simple_nodes(stations::Array{String,1}, source::String, vert_name::String, dir::String)
    name = "CI.$source Peak Amplitude Time Series at Intersect - $vert_name and B1"
    tseries = plot(title = name,linewidth=3, xtickfontsize=10, xlabel="Time", 
        ylabel="Correlation Velocity Max Amp - filt: 0.1-0.5 Hz",
        legend = :outertopright, size=(900, 400), dpi=300, leftmargin= 7mm, bottommargin=5mm)
    for (ind, file) in enumerate(stations)
        rec_name = join(split(split(file,"/")[end],".")[3:4],".")
        series = DataFrame(CSV.File(file))
        series_m = mov_avg(series, 5)
        times, amplitudes = series.date, series.max_amp
        times_m, amplitudes_m = series_m.date, series_m.moving_avg
        color=""
        if occursin("B1", rec_name)
            color = "green"
        else
            color = "blue"
        end
        println(rec_name)
        plot!(tseries, 1:length(amplitudes_m), amplitudes_m, color=color, linealpha=.6, 
                linewidth=2, label="$rec_name Mov Avg")
    end
    save_path = joinpath(plot_root,"tseries_combined/$dir/CI.$(source)_B1_$(vert_name)_intersect.png")
    if !isdir(dirname(save_path))
        mkpath(dirname(save_path))
    end
    png(save_path)
end
function wrap_plotter(vnodes::Array{String,1}, bnodes::Array{String,1}, reciever::String, vert_name, dir)
    nodes = vcat(vnodes, bnodes)
    for source_ in ["CHN","CJM","DEV","IPT","LPC","SNO","SVD","TA2"]
        try
            nodes_source = filter(x -> occursin(source_, split(x, ".")[2]), readdir("Tseries_nodes"))
            node_intersect = filter(x-> any(occursin.(join(split(x,".")[3:4],"."), nodes)), nodes_source)

            plot_simple_nodes(joinpath.("Tseries_nodes",node_intersect), source_, vert_name, dir)
            rec = "Tseries/CI.$(source_).$reciever.csv"
            println(rec)
            nodes = ["Tseries_nodes/CI.$(source_).$(node).jld2.csv" for node in nodes]
            println(nodes)
            plot_tseries(rec, joinpath.("Tseries_nodes", node_intersect), source_, dir)
        catch e
            println(e)
        end
    end
end

wrap_plotter (generic function with 1 method)

In [24]:
# List nodes at intersection with B1
G2 = ["NO.G20$elt" for elt in collect(23:27)]
G1 = ["NO.G10$elt" for elt in collect(22:27)]
G3 = ["NO.G30$elt" for elt in collect(15:17)]
G4 = ["NO.G40$(lpad(elt,2,'0'))" for elt in collect(9:13)]
B2 = ["NO.B20$elt" for elt in collect(30:37)]
B3 = ["NO.B30$elt" for elt in collect(24:29)]
B5 = ["NO.B50$elt" for elt in collect(34:38)]
B6 = ["NO.B600$elt" for elt in collect(1:5)]
B1 = ["NO.B10$elt" for elt in vcat(collect(48:52), collect(14:18), collect(86:90))]
B1_1 = ["NO.B11$(lpad(elt,2,'0'))" for elt in vcat(collect(9:12), collect(91:95))]
B1_2 = ["NO.B12$(lpad(elt,2,'0'))" for elt in vcat(collect(57:60))]
#CI.CFS = B2/B6/B1 at end, CI.FON - B4, CI.RCU - close-ish to B3, CI.PDU - near B4, CI.RIO near G1, RUS bottom of G2, CAC - kinda far from G2
recievers = ["CI.CFS", "CI.FON", "CI.RCU", "CI.PDU", "CI.RUS", "CI.CAC"] 
#nodes = vcat(G1, G2, G3, G4, B2, B3, B5, B6, B1, B1_1, B1_2)
G1_int = [G1, ["NO.B10$elt" for elt in collect(12:21)], "CI.RIO", "G1", "B1_G1_intersect"] #012-021
G2_int = [G2, ["NO.B10$elt" for elt in collect(45:60)], "CI.RUS", "G2", "B1_G2_intersect"] #045-060
G3_int = [G3, ["NO.B10$elt" for elt in collect(83:92)], "CI.RIO", "G3", "B1_G3_intersect"] #083-092
G4_int = [G4,["NO.B11$elt" for elt in collect(10:16)], "CI.PSR", "G4", "B1_G4_intersect"] #110-116

#B1_int = [B1, ["NO.B11$elt" for elt in collect(45:60)], "CI.RUS", "B1", "B1_B1_intersect"]
B2_int = [B2, ["NO.B12$elt" for elt in collect(55:65)], "CI.CFS", "B2", "B1_B2_intersect"]
B3_int = [B3, ["NO.B11$elt" for elt in collect(90:96)], "CI.RCU", "B3", "B1_B3_intersect"]
#B4_int = [B4, ["NO.B11$elt" for elt in collect(51:62)], "CI.PDU", "B4", "B1_B4_intersect"]
B5_int = [B5, ["NO.B11$elt" for elt in collect(16:24)], "CI.FON", "B5", "B1_B5_intersect"];
#B6_int = [B6, ["NO.B11$elt" for elt in collect(45:60)], "CI.CFS", "B6", "B1_B6_intersect"]

In [ ]:
for attrs in [B2_int, B3_int]#[G1_int, G2_int, G3_int, G4_int, B2_int, B3_int, B4_int, B5_int]
    wrap_plotter(attrs[1], attrs[2], attrs[3], attrs[4], attrs[5])
end